By Zeal Jinwala  <br> 
Date: June 16, 2022  <br> 
Data Source: The dataset (courtesy of Natalia Petrova) is a subset of the data used in "Prediction of catalytic residues using Support Vector Machine with selected protein sequence and structural properties", Natalia Petrova and Cathy Wu, 2006. 
http://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-7-312
Please review that publication to learn more about this dataset and the catalytic residue prediction problem.


In [59]:
from scipy.io import arff
import pandas as pd
import numpy as np
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Activation

### Load catasite data

In [67]:
data = arff.loadarff('NataliaPetrova.catsite.arff')
df = pd.DataFrame(data[0])
df['AAName1LetterCode'] = df['AAName1LetterCode'].apply(lambda x: x.decode("utf-8"))
df['class'] = df['class'].apply(lambda x: x.decode("utf-8"))
# not gonna need it but fancy code
    # needs a dictionary aa2int for all amino acids
    # AAName1LetterCode = df['AAName1LetterCode'].values
    # intAA = list(map(lambda x:aa2int[x],AAName1LetterCode))
aaOneHot = pd.DataFrame([ProteinAnalysis(i).count_amino_acids() for i in df['AAName1LetterCode']])
df1 = df[['nearest_cleft_SA_area','nearest_cleft_distance','distance_to_3_largest_clefts','HB_main_chain_protein','ScoreConsScore','class']]
result = pd.concat([aaOneHot, df1], axis=1, join="inner")

X = result.dropna()
T = X['class'].values
# X = result.iloc[:,0:25]
X.pop('class')
names = result.columns


array(['+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '+

### Train a network for classification using all features

In [58]:
kf = KFold(n_splits=4)
kf.get_n_splits(X)

for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    # print(len(X_train))
    # print(len(X_test))
    y_train, y_test = T[train_index], T[test_index]

KeyError: "None of [Int64Index([128, 129, 130, 131, 132, 133, 134, 135, 136, 137,\n            ...\n            501, 502, 503, 504, 505, 506, 507, 508, 509, 510],\n           dtype='int64', length=383)] are in the [columns]"

In [55]:
X[0:5]

array([[0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        1.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        6.571430e+02, 0.000000e+00, 0.000000e+00, 2.000000e+00,
        6.950000e-01],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 1.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        1.200658e+03, 0.000000e+00, 0.000000e+00, 1.000000e+00,
        9.010000e-01],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 1.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000

In [41]:

# Neural network model
    # 1. Define the neural network structure ( # of input units,  # of hidden units, etc). 
    # 2. Initialize the model's parameters
    # 3. Loop:
    #     - Implement forward propagation
    #     - Compute loss
    #     - Implement backward propagation to get the gradients
    #     - Update parameters (gradient descent)
X = np.asarray(X).astype(np.float32)
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X, T, epochs=150, batch_size=10)

# evaluate the keras model
_, accuracy = model.evaluate(X, T)
print('Accuracy: %.2f' % (accuracy*100))

ValueError: could not convert string to float: 'K'

### Train a network for classification using a subset of the features

### Report the average training accuracy across all repetitions.



### Report the average test accuracy across all repetitions.